# Step through and play each column of the quantum circuit

In [5]:
from circuit_functions import *
from ipywidgets import widgets
from qiskit import QuantumCircuit
import IPython

### Paste QASM code in the cell below

In [6]:
qasm = """
OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];

h q[0];
h q[1];
s q[0];
s q[1];
t q[0];
s q[1];
tdg q[0];
cx q[1],q[2];
t q[0];
s q[1];
h q[1];
cx q[0],q[1];
t q[0];
h q[1];
t q[0];
"""

circuit = QuantumCircuit.from_qasm_str(qasm)
circuit.draw()

┌───┐┌───┐┌───┐┌─────┐┌───┐          ┌───┐┌───┐
q_0: ┤ H ├┤ S ├┤ T ├┤ TDG ├┤ T ├───────■──┤ T ├┤ T ├
     ├───┤├───┤├───┤└─────┘├───┤┌───┐┌─┴─┐├───┤└───┘
q_1: ┤ H ├┤ S ├┤ S ├───■───┤ S ├┤ H ├┤ X ├┤ H ├─────
     └───┘└───┘└───┘ ┌─┴─┐ └───┘└───┘└───┘└───┘     
q_2: ────────────────┤ X ├──────────────────────────
                     └───┘                          
c: 3/═══════════════════════════════════════════════

In [7]:
class Jukebox:
    def __init__(self, circuit: QuantumCircuit):
        self.index = 0
        self.circuit = circuit
        self.sub_circuits = None
        self.state_vectors = None
        self.buttons = self.get_buttons()
            
        self.load_circuit(circuit)
        
        # Display UI controls
        self.display()
    
    def clear_output(self):
        """Clears all output except the button controls"""
        IPython.display.clear_output(wait=True)
        self.display()
    
    def load_circuit(self, circuit: QuantumCircuit):
        """Overwrite the current circuit with the new circuit"""
        self.sub_circuits = get_circuits_by_column(circuit)
        self.state_vectors = get_cummulative_state_vectors(self.sub_circuits)
    
    def play(self, button):
        self.clear_output()
        notes = get_notes(self.state_vectors[self.index])
        play_notes(notes)
        
    def play_all_from(self, button):
        for i in range(self.index, len(self.state_vectors)):
            self.clear_output()
            self.index = i
            self.play(button)
            sleep(1)
        
    def restart(self, button):
        """Moves back to the first column"""
        if self.index == 0:
            return
        self.clear_output()
        self.index = 0
        print(f'Column={self.index}')
        
    def back(self, button):
        if self.index == 0:
            return
        self.clear_output()
        self.index -= 1
        self.play(button)
        print(f'Column={self.index}')

    def forward(self, button):
        if self.index == len(self.sub_circuits) - 1:
            return
        self.clear_output()
        self.index += 1
        self.play(button)
        print(f'Column={self.index}')
    
    def get_buttons(self):
        restart_button = widgets.Button(icon='fast-backward', tooltip='Restart to beginning')
        restart_button.on_click(self.restart)
        
        back_button = widgets.Button(icon='backward')
        back_button.on_click(self.back)
        
        play_button = widgets.Button(icon='play')
        play_button.on_click(self.play)
        
        play_all_from_button = widgets.Button(icon='play-circle', tooltip='Automatically play to the end')
        play_all_from_button.on_click(self.play_all_from)

        forward_button = widgets.Button(icon='forward')
        forward_button.on_click(self.forward)
        
        return [
            restart_button,
            back_button,
            play_button,
            play_all_from_button,
            forward_button
        ]
        
    def display(self):
        """Display the audio controls UI"""
        # Display in HBox
        display(self.circuit.draw())
        display(self.sub_circuits[self.index].draw())
        buttons = widgets.HBox(self.buttons)
        display(buttons)

In [8]:
jukebox = Jukebox(circuit)

┌───┐┌───┐┌───┐┌─────┐┌───┐          ┌───┐┌───┐
q_0: ┤ H ├┤ S ├┤ T ├┤ TDG ├┤ T ├───────■──┤ T ├┤ T ├
     ├───┤├───┤├───┤└─────┘├───┤┌───┐┌─┴─┐├───┤└───┘
q_1: ┤ H ├┤ S ├┤ S ├───■───┤ S ├┤ H ├┤ X ├┤ H ├─────
     └───┘└───┘└───┘ ┌─┴─┐ └───┘└───┘└───┘└───┘     
q_2: ────────────────┤ X ├──────────────────────────
                     └───┘                          
c: 3/═══════════════════════════════════════════════

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘
q_2: ─────
          
c: 3/═════